# Detect house damage or associated risk from video

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# importing libraries
import cv2
import os
import shutil
from fastai import *
from fastai.vision import *
#! pip install pytube
from pytube import YouTube
#! conda install -c conda-forge imageio
import imageio
#! pip install moviepy
from moviepy.editor import *
from ipywidgets import Video

## Download youtube video

In [102]:
filepath = 'C:/Users/manas/data/Video'
url = 'https://www.youtube.com/watch?v=WgrL79xPOYo'
save_fn = 'Open House Tour1'
yt = YouTube(url).streams.first().download(output_path=filepath, filename=save_fn)

## Read Video, save to images and annotate them

### Methodology
I have combined 8 models from different modeling teams to detect any damage/risk in a house. For the dead lanscape model I have retrained the model and included a none category. The accuracy received is 0.89. Time permitted, I could have increased accuracy of this model.

The steps of the project are as follows:

Step 1: Detect childproof fall risk classes: balcony, bathtub, stairs, swimming pool, window, none. If the detected category is 'none' write nothing on image.

Step 2: Detect (dead) landscape type, e.g., dead, healthy or none. If the detected category is 'none', apply exterior building damage model. If the detected category is healthy, apply overgrown landscape model

Step 3: If the image category (clean, overgrown) using overgrown landscape model is 'clean' write nothing on image.

Step 4: If the image category using exterior building damage is 'none' implement these models: wall stain model, furniture stain model, childproof heavy object model, childproof electrical model. These models would run in parallel. If the detected category from those models are 'safe' or 'clean' those would not be written on the images.

In [69]:
#vidcap = cv2.VideoCapture('C:/Users/manas/data/building_data/Southwest Florida Home Inspection 10.25-10.40.mp4')
vidcap = cv2.VideoCapture('C:/Users/manas/data/Video/Open House Tour1.mp4')

In [70]:
frame_folder_path = 'C:/Users/manas/data/video_frames/Open_house_tour_1/'

In [71]:
# function to run for getting rid of attribute error
from torch.utils.data.sampler import WeightedRandomSampler
class OverSamplingCallback(LearnerCallback):
   def __init__(self,learn:Learner,weights:torch.Tensor=None):
       super().__init__(learn)
       self.labels = self.learn.data.train_dl.dataset.y.items
       _, counts = np.unique(self.labels,return_counts=True)
       self.weights = (weights if weights is not None else
                       torch.DoubleTensor((1/counts)[self.labels]))
       self.label_counts = np.bincount([self.learn.data.train_dl.dataset.y[i].data for i in range(len(self.learn.data.train_dl.dataset))])
       self.total_len_oversample = int(self.learn.data.c*np.max(self.label_counts))
   def on_train_begin(self, **kwargs):
       self.learn.data.train_dl.dl.batch_sampler = BatchSampler(WeightedRandomSampler(self.weights,self.total_len_oversample), self.learn.data.train_dl.batch_size,False)

In [72]:
# loading learner objects
learn_cf = load_learner('C:/Users/manas/data/childproofing_fall')
learn_dl = load_learner('C:/Users/manas/data/dead_landscape_model')
learn_ol = load_learner('C:/Users/manas/data/overgrown_landscape')
learn_ol.to_fp32()
learn_ed = load_learner('C:/Users/manas/data/building_data')
learn_ws = load_learner('C:/Users/manas/data/wall_stain')
learn_fs = load_learner('C:/Users/manas/data/furniture_stain')
learn_ch = load_learner('C:/Users/manas/data/childproffing_heavy_object')
learn_ce = load_learner('C:/Users/manas/data/childproofing_electrical')

In [73]:
def bldg_detect(learn, frame_path):
    prediction = learn.predict(open_image(frame_path))
    category = prediction[0]
    probability = "{0:.4f}".format(max(prediction[2]))
    return category, probability    

In [ ]:
filepath = 'C:/Users/manas/data/video_frames_text/Open_house_tour1/'
count = 0
success = 1
while success:
    success,frame = vidcap.read()
    frame_path = frame_folder_path + f'frame{count}.jpg'
    cv2.imwrite(frame_path,frame)
    #print(frame_path)
    
    # model prediction: childproof fall
    category_cf, probability_cf = bldg_detect(learn_cf,frame_path)
    if str(category_cf) =='none':
        category_fall = ''
    else:
        category_fall = 'fall risk ' + str(category_cf)
        
        
    # model prediction: dead landscape
    category_dl, probability_dl = bldg_detect(learn_dl,frame_path)
    if str(category_dl) =='none':
        category_l = ''
        #model prediction: exterior damage
        category_ed, probability_ed = bldg_detect(learn_ed,frame_path)    

        if str(category_ed) == 'none':
            # model prediction for wall stain
            category_ws, probability_ws = bldg_detect(learn_ws,frame_path)
            if str(category_ws) == 'clean':
                category_wall = ''
            else:
                category_wall = 'wall ' + str(category_ws)
            # model prediction for furniture stain    
            category_fs, probability_fs = bldg_detect(learn_fs,frame_path)
            if str(category_fs) == 'clean':
                category_furn = ''
            else:
                category_furn = 'furniture ' + str(category_fs)

            # model prediction for heavyobject risk
            category_ch, probability_ch = bldg_detect(learn_ch,frame_path)
            if str(category_ch) == 'safe_furniture' or str(category_ch) == 'safe_kitchen' or str(category_ch) == 'safe_tv' or str(category_ch) == 'safe_furniture;safe_kitchen' or str(category_ch) == 'safe_kitchen;safe_furniture' or str(category_ch) == 'safe_furniture;safe_tv' or str(category_ch) == 'safe_tv;safe_furniture':
                category_heavy = ''
            
            else:
                category_heavy = str(category_ch)

            # model prediction for electrical risk
            category_ce, probability_ce = bldg_detect(learn_ce,frame_path)
            if str(category_ce) == 'childproofed':
                category_electric = ''
            else:
                category_electric = 'electrical - ' + str(category_ce)

            #writing to category building landscape and building risk
            #category_l = str(category_wall) +'| ' + str(category_furn) +'| ' + str(category_heavy) + '| '+ str(category_electric)
            if str(category_wall) != '':
                category_l = category_l + str(category_wall) 
            if str(category_furn) != '':
                if category_l != '':
                    category_l = category_l + ' | ' 
                category_l =category_l + str(category_furn) 
   
            if str(category_heavy) != '':
                if category_l != '':
                    category_l = category_l + ' | ' 
                category_l =category_l + str(category_heavy) 

            if str(category_electric) != '':
                if category_l != '':
                    category_l = category_l + ' | '     
                category_l =category_l + str(category_electric) 
                
        else:
            category_l = str(category_ed)
    else:
        category_l = 'landscape ' + str(category_dl)
    
    # model prediction for overgrown landscape
    if str(category_dl) == "healthy":
        category_ol, probability_ol = bldg_detect(learn_ol,frame_path)
        if str(category_ol) =='clean':
            category_l = ''
        else:
            category_l = 'landscape ' + str(category_ol)
    
    # writing to category child fall risk, building landscape and building risk
    #category = str(category_fall) + '| ' + str(category_l)
    
    category = ''
    
    if str(category_fall) != '':
        category = str(category_fall)
        
    if str(category_l) != '':
        if category != '':
            category = category + ' | ' + str(category_l)
        else:
            category = str(category_l)
    
    
    # writing category text on image 
    
    if category != '':
        #text = f'{category} \n {probability}'
        text = f'{category}'

        #Writing on iamge
        font = cv2.FONT_HERSHEY_SIMPLEX

        # set the rectangle background to white
        rectangle_bgr = (255, 255, 255)
        # get the width and height of the text box
        (text_width, text_height) = cv2.getTextSize(text, font, fontScale=1.0, thickness=1)[0]
        # set the text start position
        text_offset_x = 5
        text_offset_y = 16
        # make the coords of the box with a small padding (change  coordinate to adjust)
        box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 10, text_offset_y + text_height + 10))
        cv2.rectangle(frame, box_coords[0], box_coords[1], rectangle_bgr, cv2.FILLED)

        
        img_with_text = cv2.putText(frame,text,(10, 40),
            font,1,(255,0,0),2,cv2.LINE_AA)

        # Save the image

        cv2.imwrite(filepath + f'frame{count}.jpg',img_with_text)

    else:
        cv2.imwrite(filepath + f'frame{count}.jpg',frame)

    cv2.imshow('frame',frame)
    cv2.waitKey(1)
    count+=1

## From image to video

In [75]:
import cv2
import numpy as np
import os
from os.path import isfile, join

In [76]:
pathIn= 'C:/Users/manas/data/video_frames_text/Open_house_tour1/'
pathOut = 'C:/Users/manas/data/Video/Open_house_tour1_text.mp4'
fps = 20
frame_array = []
files = [f for f in os.listdir(pathIn)]

for i in range(len(files)):
    filename=pathIn + 'frame'+str(i+8)+ '.jpg'
    
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
    
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()


## Conclusion

Many models including ours fall short in classifying the image properly. The main problem lies on how the models were trained individually earlier. For example, our exterior bulding damage model, - although we had several damage categories and a 'none' category, we included mostly undamaged exterior building parts in the trainset, and no building interior images. So, the model sometime fails to classify the interior images correctly. Also, many rotten window frames were of white colour which creates bias in classification. Sometimes reflections on window are classified as broken window glass. We need to have much larger dataset with different image types to train a better model. I could have included other models, e.g., water damage, structural damage etc to make this model ensemble even more useful. 

Other observations from several video run-
1. Photoframes, rectangular shaped stuffs on shelves, fileplace are often classified as tv (childproof heavy object). Those should be included in 'none' category.

2. Anything portruding from wall / wall-like is classified as open-wire (childproof electrical). Those should be included in a  'none' catgory too.

3. Switchboard, and small decors with holes are sometimes missclassified as open plug (childproof electrical).

4. Some designs on furniture are missclassified as 'stain' or 'distress' . This is same with wall decor, wall texture or any design on wall (wall stain, furniture stain).

5. Organized fruits/objects on table are sometimes categorized as part of kitchen. It would be useful to include a 'none' category for childproof heavy object model.

6. Object with a particluar shape, e.g., basin, sink etc are sometimes classified as bathtub (childproof fall). 

Including 'none' category to each model training set would be helpful for designing this model ensemble. The 'none' category should contain all other type of images associated with a building apart from the interest category.

Next, I want to implement gradcam in this model to show where the model is focusing for classification.

The video is uploaded in youtube: https://www.youtube.com/watch?v=_aIYhrGMioo&t=23s